# Regressão Logística Multinomial para Classificação de Peso em Meninas de 15 Anos
Este notebook implementa um modelo de regressão logística multinomial para classificar meninas de 15 anos nas categorias: obesidade, sobrepeso, peso normal e anorexia.

In [ ]:
# Importar bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Gerar dados sintéticos para meninas de 15 anos
np.random.seed(42)
n_samples = 1000

# Definir parâmetros para cada categoria baseados em dados médicos
# IMC categorias: Anorexia (<16), Normal (18.5-24.9), Sobrepeso (25-29.9), Obesidade (>=30)

# Anorexia: altura normal, peso muito baixo
altura_anorexia = np.random.normal(160, 8, 100)  # cm
peso_anorexia = np.random.normal(40, 3, 100)     # kg

# Peso normal
altura_normal = np.random.normal(162, 7, 400)    # cm
peso_normal = np.random.normal(55, 5, 400)       # kg

# Sobrepeso
altura_sobrepeso = np.random.normal(161, 6, 300) # cm
peso_sobrepeso = np.random.normal(70, 6, 300)    # kg

# Obesidade
altura_obesidade = np.random.normal(159, 7, 200) # cm
peso_obesidade = np.random.normal(85, 8, 200)    # kg

# Combinar os dados
altura = np.concatenate([altura_anorexia, altura_normal, altura_sobrepeso, altura_obesidade])
peso = np.concatenate([peso_anorexia, peso_normal, peso_sobrepeso, peso_obesidade])

# Calcular IMC
imc = peso / ((altura/100) ** 2)

# Definir categorias
categorias = ['Anorexia'] * 100 + ['Normal'] * 400 + ['Sobrepeso'] * 300 + ['Obesidade'] * 200

# Criar DataFrame
df = pd.DataFrame({
    'altura_cm': altura,
    'peso_kg': peso,
    'imc': imc,
    'categoria': categorias
})

print("Dataset gerado com sucesso!")
print(f"Tamanho do dataset: {len(df)} amostras")
print("\nDistribuição das categorias:")
print(df['categoria'].value_counts())

In [ ]:
# Explorar os dados
print("Estatísticas descritivas:")
print(df.describe())
print("\nPrimeiras 10 linhas:")
print(df.head(10))

In [ ]:
# Visualização dos dados
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribuição do IMC por categoria
sns.boxplot(data=df, x='categoria', y='imc', ax=axes[0,0])
axes[0,0].set_title('Distribuição do IMC por Categoria')
axes[0,0].tick_params(axis='x', rotation=45)

# Scatter plot Altura vs Peso
sns.scatterplot(data=df, x='altura_cm', y='peso_kg', hue='categoria', ax=axes[0,1], alpha=0.7)
axes[0,1].set_title('Altura vs Peso por Categoria')

# Distribuição das categorias
df['categoria'].value_counts().plot(kind='bar', ax=axes[1,0], color='skyblue')
axes[1,0].set_title('Distribuição das Categorias')
axes[1,0].tick_params(axis='x', rotation=45)

# Histograma do IMC
axes[1,1].hist(df['imc'], bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1,1].set_title('Distribuição do IMC')
axes[1,1].set_xlabel('IMC')
axes[1,1].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

In [ ]:
# Preparar dados para o modelo
# Features: altura, peso e IMC
X = df[['altura_cm', 'peso_kg', 'imc']].values
y = df['categoria'].values

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Padronizar as features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")
print(f"Número de features: {X_train.shape[1]}")

In [ ]:
# Treinar modelo de Regressão Logística Multinomial
log_reg = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    random_state=42,
    max_iter=1000
)

log_reg.fit(X_train_scaled, y_train)

print("Modelo treinado com sucesso!")
print(f"Classes identificadas: {log_reg.classes_}")
print(f"Número de iterações: {log_reg.n_iter_}")

In [ ]:
# Fazer predições
y_pred = log_reg.predict(X_test_scaled)
y_pred_proba = log_reg.predict_proba(X_test_scaled)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.3f}")

print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred))

In [ ]:
# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=log_reg.classes_, 
            yticklabels=log_reg.classes_)
plt.title('Matriz de Confusão')
plt.xlabel('Predição')
plt.ylabel('Valor Real')
plt.show()

In [ ]:
# Visualizar coeficientes do modelo
feature_names = ['Altura (cm)', 'Peso (kg)', 'IMC']
coef_df = pd.DataFrame(
    log_reg.coef_.T, 
    index=feature_names, 
    columns=log_reg.classes_
)

plt.figure(figsize=(10, 6))
sns.heatmap(coef_df, annot=True, cmap='RdBu_r', center=0, fmt='.2f')
plt.title('Coeficientes do Modelo de Regressão Logística')
plt.xlabel('Categorias')
plt.ylabel('Features')
plt.show()

print("Coeficientes do modelo:")
print(coef_df)

In [ ]:
# Função para classificar uma nova menina de 15 anos
def classificar_menina(altura_cm, peso_kg, modelo=log_reg, scaler=scaler):
    """
    Classifica uma menina de 15 anos baseada na altura e peso.
    
    Args:
        altura_cm: altura em centímetros
        peso_kg: peso em quilogramas
        modelo: modelo treinado
        scaler: scaler ajustado
    
    Returns:
        categoria prevista e probabilidades
    """
    imc = peso_kg / ((altura_cm/100) ** 2)
    
    # Preparar entrada
    entrada = np.array([[altura_cm, peso_kg, imc]])
    entrada_scaled = scaler.transform(entrada)
    
    # Predição
    categoria = modelo.predict(entrada_scaled)[0]
    probabilidades = modelo.predict_proba(entrada_scaled)[0]
    
    print(f"Menina de 15 anos:")
    print(f"Altura: {altura_cm} cm")
    print(f"Peso: {peso_kg} kg")
    print(f"IMC: {imc:.2f}")
    print(f"\nCategoria prevista: {categoria}")
    print(f"\nProbabilidades:")
    for classe, prob in zip(modelo.classes_, probabilidades):
        print(f"  {classe}: {prob:.3f} ({prob*100:.1f}%)")
    
    return categoria, probabilidades

# Exemplos de classificação
print("Exemplo 1: Menina com possível anorexia")
classificar_menina(165, 42)

print("\n" + "="*50 + "\n")

print("Exemplo 2: Menina com peso normal")
classificar_menina(160, 55)

print("\n" + "="*50 + "\n")

print("Exemplo 3: Menina com sobrepeso")
classificar_menina(158, 68)

print("\n" + "="*50 + "\n")

print("Exemplo 4: Menina com obesidade")
classificar_menina(155, 80)

## Resumo dos Resultados

### Modelo Implementado
- **Algoritmo**: Regressão Logística Multinomial
- **Features**: Altura, Peso e IMC
- **Classes**: Anorexia, Normal, Sobrepeso, Obesidade

### Características do Modelo
1. **Classificação Multiclasse**: Utiliza a abordagem "one-vs-rest" para classificar em 4 categorias
2. **Probabilidades**: Fornece probabilidades para cada categoria
3. **Interpretabilidade**: Os coeficientes mostram a importância de cada feature

### Aplicações
- Triagem inicial em consultas médicas
- Monitoramento de saúde em escolas
- Suporte a profissionais de saúde

### Limitações
- Baseado apenas em medidas antropométricas
- Não considera fatores genéticos ou metabólicos
- Requer validação com dados reais

**Nota**: Este modelo é apenas para fins educacionais. Decisões médicas reais devem sempre envolver profissionais qualificados.